In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import pandas as pd

### 1) Elaborar un dataset de preguntas y respuestas para crear un Chatbot para un aplicación particular.

In [2]:
df=pd.read_excel('preguntas-respuestas.xlsx')
df.head(3)

,Pregunta,Respuesta
0,¿Como puedo solicitar o hacer seguimiento de m...,Es posible que tengas derecho al reembolso. Si...
1,¿Cuál es el equipaje permitido?,Por cuestiones de seguridad existen ciertas li...
2,¿Qué reservas puedo cambiar online?,Se pueden realizar cambios de fecha en todas l...


###  2) Crear el chatbot utilizando TFIDF y similitud del coseno.

In [3]:
vectorizer_agencia = TfidfVectorizer()
tfidf_matrix = vectorizer_agencia.fit_transform(df['Pregunta'])

In [4]:
def responder_pregunta(consulta_usuario):
    consulta_vec = vectorizer_agencia.transform([consulta_usuario])
    similitudes = cosine_similarity(consulta_vec, tfidf_matrix).flatten()
    print(similitudes)
    indice_mejor_coincidencia = similitudes.argmax()
    print(indice_mejor_coincidencia)
    return df['Respuesta'][indice_mejor_coincidencia]
 

In [5]:
consulta = "¿Qué reservas es posible cambiar de fecha?"
print(responder_pregunta(consulta))

[0.11854842 0.18232138 0.62664211 0.10340829 0.14637087 0.05650839
 0.21394124 0.07562619 0.06366248 0.         0.        ]
2
Se pueden realizar cambios de fecha en todas las reservas con tramos de ida o ida y vuelta excepto tramos multidestinos generadas en aerolineas.com. En nuestras sucursales y Call Center (excepto tickets emitidos en Brasil y Chile).


### 3) Crear otro chatbot utilizando embeddings. Indique cuál embedding pre-entrenado eligió.

In [6]:
nlp = spacy.load("es_core_news_md")

In [7]:
preguntas= [nlp(pregunta) for pregunta in df['Pregunta']]

In [8]:
def consul_emb(consulta):
    consulta=nlp(consulta)
    similitudes = [pregunta.similarity(consulta) for pregunta in preguntas]
    print(similitudes)
    mejor_respuesta = df['Respuesta'][similitudes.index(max(similitudes))]
    return mejor_respuesta

In [9]:
consulta="¿Qué animales se pueden llevar en bodega?"
print(consul_emb(consulta))

[0.5867151351517318, 0.5737322659452583, 0.706471439796241, 0.6126360769339564, 0.7012249586272455, 0.8045942964889908, 0.6758986861898805, 0.8651269787486385, 0.5486669945470013, 0.5952350091821499, 0.664373743416445]
Puedes llevar animales domesticos. Reservá el servicio 72 horas antes de la salida de tu vuelo por WhatsApp +54 911 4940-4798 o al Contact Center +54 (11) 5199 3555.
